In [12]:
import numpy as np
import os
import time
import glob
import matplotlib.pyplot as plt
from numba import jit, njit
from matplotlib import ticker
import pandas as pd



def load_position_2D(position_name="../data/position"):
	"""Load 2-dimensional position/trajectory data.

	Args:
		position_name (str, optional): Name of position file. Defaults to "position".

	Returns:
		list: 2 * np.array with position data of each dimension ([position_x, position_y])
	"""
	for file1 in glob.glob(position_name):
		colvar = np.loadtxt(file1)
		position_x = colvar[:-1, 1]
		position_y = colvar[:-1, 2]
	return [position_x, position_y]


#Define grid 
min_grid=np.array((-np.pi, -np.pi))
max_grid=np.array((np.pi, np.pi))
nbins=np.array((200, 200))
gridy = np.linspace(min_grid[1], max_grid[1], nbins[1])
gridx = np.linspace(min_grid[0], max_grid[0], nbins[0])
grid_space = np.array(((max_grid[0] - min_grid[0]) / (nbins[0]-1), (max_grid[1] - min_grid[1]) / (nbins[1]-1)))
X, Y = np.meshgrid(gridx, gridy)
#Define constants
bw = 0.1
bw2 = bw**2
stride = 10
const = (1 / (bw * np.sqrt(2 * np.pi) * stride))
kT = 1
x0, y0 = 0,0


[p_x,p_y] = load_position_2D("../data/position40")

nbins=np.array((200, 200))
results = np.zeros((3,nbins[0],nbins[1]))
stride = 10

def gaus_pandas(data_x, data_y):
    pb_t = np.zeros_like(X)
    Fpbt_x = np.zeros_like(X)
    Fpbt_y = np.zeros_like(X)
    df = pd.DataFrame({'data_x': data_x, 'data_y': data_y})
    kernel = const * np.exp(- (1 / (2 * bw2)) * ((X - df['data_x']) ** 2 + (Y - df['data_y']) ** 2))
    pb_t = kernel.sum()
    Fpbt_x = (kernel * kT * (X - df['data_x']) / bw2).sum()
    Fpbt_y = (kernel * kT * (Y - df['data_y']) / bw2).sum()
    return [pb_t, Fpbt_x, Fpbt_y]

In [13]:
n_eval=1
for i in range(n_eval): results += gaus_pandas(p_x[i*stride: (i+1)*stride],p_y[i*stride: (i+1)*stride])




ValueError: operands could not be broadcast together with shapes (200,200) (10,) 